In [ ]:
#Importa librerias necesarias
import time
import pandas as pd
from selenium import webdriver
from pymongo import MongoClient

In [ ]:
#Asigna la url de accion trabajo filtrando desde los mas recientes
url = "https://acciontrabajo.ec/buscar-empleos?o=d"

In [ ]:
#Conecta con mongodb
Client = MongoClient('localhost',27017)

In [ ]:
#Lista las bases de mongodb
Client.list_database_names()

In [ ]:
#Selecciona la base alldata
db = Client.alldata
db

In [ ]:
#Lista las colecciones
colecciones = db.list_collection_names()
colecciones

In [ ]:
#Selecciona la coleccion creada para acciontrabajo
collection = db["acciontrabajo"]
collection

In [ ]:
#Crea el metodo
def extraer_ofertas(driver):
    print('Realizando scraping...')
    #inicia desde la oferta mas reciente
    i=0
    try:
        #guarda las ofertas en una lista
        ofertas = driver.find_elements_by_xpath('//*[@id="main_content"]/div/div/div/article/section')
        #recorre la lista
        while i <= len(ofertas):
            #vuelve a guardar la lista de las ofertas para actualizarla
            ofertas = driver.find_elements_by_xpath('//*[@id="main_content"]/div/div/div/article/section')
            #selecciona la oferta deacuerdo a la posicion del while
            oferta=ofertas[i]
            #obtiene una parte de los datos
            cargo = oferta.find_element_by_xpath('./div[@class="list_item v_list_item"]/a/h2').text
            fechapublicacion = oferta.find_element_by_xpath('./div[@class="list_item v_list_item"]//div[@class="basic_details"]/div/div').text
            #clickea la oferta
            oferta.find_element_by_xpath('./div[@class="list_item v_list_item"]/a').click()
            #agrega un tiempo de demora
            time.sleep(5)
            #obtiene la otra parte de los datos
            ciudad = driver.find_element_by_xpath('//*[@id="core_content"]/div[1]/div/div[2]/div/div[1]/div[2]/span').text 
            tipocontrato = driver.find_element_by_xpath('//*[@id="core_content"]/div[1]/div/div[2]/div/div[4]/div[2]').text
            experiencia = driver.find_element_by_xpath('//*[@id="core_content"]/div[1]/div/div[2]/div/div[2]/div[2]').text[12:]
            remuneracion = driver.find_element_by_xpath('//*[@id="core_content"]/div[1]/div/div[2]/div/div[5]/div[2]').text[8:]
            areaestudios = driver.find_element_by_xpath('//*[@id="core_content"]/div[1]/div/div[2]/div/div[3]').text
            contenido = driver.find_element_by_xpath('//*[@id="core_content"]/div[2]/div/div').text.replace('\n',"")
            
            #guarda en un diccionario
            oferta_dict = {
                'portalweb':"acciontrabajo",
                'cargo':cargo,
                'tipocontrato':tipocontrato,
                'ciudad':ciudad,
                'fechapublicacion':fechapublicacion,
                'remuneracion':remuneracion,
                'areaestudios':areaestudios,
                'experiencia':experiencia,
                'contenido':contenido
            }
            
            #consulta si la oferta ya existe
            if collection.find_one(oferta_dict):
                #si existe regresa
                driver.back()
            else:
                #si no existe la inserta en la coleccion 
                collection.insert_one(oferta_dict)
                #regresa
                driver.back()
            #agrega un tiempo de demora
            time.sleep(10)
            i+= 1
    except:
        print('Terminado')
            

In [ ]:
#Inicia el web driver desde su ubicacion en el equipo
driver = webdriver.Chrome(executable_path='C:/Users/DELL/Downloads/chromedriver.exe')

In [ ]:
#Ingresa a la url
driver.get(url)

In [ ]:
#Ejecuta el metodo
extraer_ofertas(driver)